# Exploring Russell Humphreys geomagnetism and water connection

In [1]:
from astropy.constants import M_earth     # mass of the Earth
from astropy.constants import M_jup       # mass of Jupiter
from astropy.constants import M_sun       # mass of the Sun
from astropy.constants import m_e         # mass of an electron
from astropy.constants import m_n         # mass of a neutron
from astropy.constants import m_p         # mass of a proton
from astropy.constants import R_earth     # radius of the Earth
from astropy.constants import N_A         # Avogadro's number
from astropy.constants import u           # Atomic mass
from astropy.constants import mu0         # vacuum magnetic permeability
from astropy.constants import eps0        # vacuum electric permittivity
from astropy.constants import Constant    # constant constructor
from astropy.constants import EMConstant  # electromagnetic constant constructor
import astropy.units as u
from astropy.units import Quantity
from astropy.table import QTable
from astropy.time import Time
from astropy.nddata import NDData
import typing as T
from astropy import find_api_page
import numpy as np
import math
import datetime

In [2]:
"""
The proton magnetic moment I obtained from
https://physics.nist.gov/cuu/pdf/wall_2018.pdf
and defined it using astropy.constants EMConstant.
"""

muP = EMConstant(
    abbrev      = "muP", 
    name        = "Proton magnetic moment", 
    value       = 1.41060679736e-26, 
    unit        = u.J / u.T, 
    uncertainty = 0.00000000060e-26, 
    reference   = "CODATA 2018", 
    system      = "si"
)

"""
The atomic mass of an oxygen atom.
https://physics.nist.gov/cgi-bin/Elements/elInfo.pl?element=8
"""

m_O = Constant(
    abbrev      = "m_O",
    name        = "Oxygen atomic mass",
    value       = 15.99940,
    unit        = u.g / u.mol,
    uncertainty = 0.00037,
    reference   = "NIST https://physics.nist.gov/cgi-bin/Elements/elInfo.pl?element=8"
)

"""
The atomic mass of a hydrogen atom.
https://physics.nist.gov/cgi-bin/Elements/elInfo.pl?element=1
"""

m_H = Constant(
    abbrev      = "m_H",
    name        = "Hydrogen atomic mass",
    value       = 1.007975,
    unit        = u.g / u.mol,
    uncertainty = 0.000135,
    reference   = "NIST https://physics.nist.gov/cgi-bin/Elements/elInfo.pl?element=1"
)

M_water = Constant(
    abbrev      = "M_water",
    name        = "Mass of a water molecule",
    value       = ((2 * m_H + m_O) / N_A).to(u.kg),
    unit        = u.kg,
    uncertainty = 0,
    reference   = "Derived from m_H, m_O and N_A"
)

"""
The radius of the Earth's core.
"""

R_earth_core = Constant(
    abbrev      = "R_earth_core",
    name        = "Radius Earth's Core",
    value       = 3471,
    unit        = u.km,
    uncertainty = 0,
    reference   = "D. Russell Humphrey"
)

"""
The following are constants used in Humphreys' papers which I will use to check the algorithms are correct
and to verify that Humphreys' data is correct.

All of thesse constants can be identified with a final "_H" in their abbreviation
"""

R_earth_H = Constant(
    abbrev      = "R_earth_H",
    name        = "Humphreys' Earth radius",
    value       = 6371.2,
    unit        = u.km,
    uncertainty = 0,
    reference   = "D. Russell Humphreys, The Earth's Magnetic Field is Still Losing Energy, CRSQ 39, 2002, p.6"
)

R_earth_core_H = Constant(
    abbrev      = "R_earth_core_H",
    name        = "Humphreys' Earth radius core",
    value       = 3471,
    unit        = u.km,
    uncertainty = 0,
    reference   = "D. Russell Humphreys, The Earth's Magnetic Field is Still Losing Energy, CRSQ 39, 2002, p.8"
)

mu0_H = Constant(
    abbrev      = "mu0_H",
    name        = "Humphreys' vacuum magnetic permeability",
    value       = 4 * math.pi * 1e-7,
    unit        = u.henry / u.meter,
    uncertainty = 0,
    reference   = "D. Russell Humphreys, The Creation of Earth's Magnetic Field, CRSQ 20, 1983, p.92"
)


In [3]:
# Displaying all of the constants

for i in (M_earth, M_jup, M_sun, m_e, m_n, m_p, R_earth, R_earth_H, R_earth_core, R_earth_core_H, N_A, mu0, mu0_H, eps0, muP, m_O, m_H, M_water):
    print(i.abbrev)
    print(i)
    print()

M_earth
  Name   = Earth mass
  Value  = 5.972167867791379e+24
  Uncertainty  = 1.3422009501651213e+20
  Unit  = kg
  Reference = IAU 2015 Resolution B 3 + CODATA 2018

M_jup
  Name   = Jupiter mass
  Value  = 1.8981245973360505e+27
  Uncertainty  = 4.26589589320839e+22
  Unit  = kg
  Reference = IAU 2015 Resolution B 3 + CODATA 2018

M_sun
  Name   = Solar mass
  Value  = 1.988409870698051e+30
  Uncertainty  = 4.468805426856864e+25
  Unit  = kg
  Reference = IAU 2015 Resolution B 3 + CODATA 2018

m_e
  Name   = Electron mass
  Value  = 9.1093837015e-31
  Uncertainty  = 2.8e-40
  Unit  = kg
  Reference = CODATA 2018

m_n
  Name   = Neutron mass
  Value  = 1.67492749804e-27
  Uncertainty  = 9.5e-37
  Unit  = kg
  Reference = CODATA 2018

m_p
  Name   = Proton mass
  Value  = 1.67262192369e-27
  Uncertainty  = 5.1e-37
  Unit  = kg
  Reference = CODATA 2018

R_earth
  Name   = Nominal Earth equatorial radius
  Value  = 6378100.0
  Uncertainty  = 0.0
  Unit  = m
  Reference = IAU 2015 Reso

In [4]:
# Derived quantities
muW = 2 * muP                              # Water magnetic moment
#M_water = ((2 * m_H + m_O) / N_A).to(u.kg) # Water mass, see https://sciencenotes.org/calculate-the-mass-in-grams-of-a-single-water-molecule/
#n = M_earth / M_water                      # Number of water molecules equivalent to Earth's mass

"""
k is the fraction of hydrogen atoms which are aligned.  It ranges from 0 to 1.
n is the number of water molecules if the entire mass of the Earth were water.

Return: The magnetic moment of that fraction of water.
"""

#def mu_earth_creation(k):
#    return 2 * k * n * muP

def mu_creation(k, mass):
    return 2 * k * (mass / M_water) * muP

"""
k is the fraction of hydrogen atoms which are aligned.  It ranges from 0 to 1.
mu is the current magnetic moment of the Earth.
BC is the year BC when creation occurred.

Return: The rate of decay of the magnetic field.
"""

def rate_decay(k, mu, mass, BC):
    original_mu = mu_creation(k, mass)
    return - (BC + datetime.date.today().year - 1) / (math.log(mu, math.e) - math.log(original_mu.value, math.e))
    


In [5]:
current_mu = 7.9e22
r25_4004 = rate_decay(0.25, current_mu, M_earth, 4004)
r25_4004

2092.3386644988973

In [6]:
# https://creation.com/biblical-age-of-the-earth

a = np.array([0.25, 
              0.50, 
              0.75, 
              1.00]
            )
b = [mu_creation(0.25, M_earth), 
     mu_creation(0.50, M_earth), 
     mu_creation(0.75, M_earth), 
     mu_creation(1, M_earth)
    ] 
current_mu = 7.9e22
c = [rate_decay(0.25, current_mu, M_earth, 3822), 
     rate_decay(0.50, current_mu, M_earth, 3822), 
     rate_decay(0.75, current_mu, M_earth, 3822), 
     rate_decay(1, current_mu, M_earth, 3822)
    ]
d = [rate_decay(0.25, current_mu, M_earth, 4004), 
     rate_decay(0.50, current_mu, M_earth, 4004), 
     rate_decay(0.75, current_mu, M_earth, 4004), 
     rate_decay(1, current_mu, M_earth, 4004)
    ]
e = [rate_decay(0.25, current_mu, M_earth, 5554), 
     rate_decay(0.50, current_mu, M_earth, 5554), 
     rate_decay(0.75, current_mu, M_earth, 5554), 
     rate_decay(1, current_mu, M_earth, 5554)
    ]
f = [rate_decay(0.25, current_mu, M_earth, 5665), 
     rate_decay(0.50, current_mu, M_earth, 5665), 
     rate_decay(0.75, current_mu, M_earth, 5665), 
     rate_decay(1, current_mu, M_earth, 5665)
    ]

t = QTable([a, b, c, d, e, f],
           names=('k', 
                  'Creation Magnetic Moment', 
                  '3822 BC', 
                  '4004 BC', 
                  '5554 BC', 
                  '5665 BC'
                 ),
           meta={'name': 'first table'})
t['k'].info.format = '1.2f'
t['Creation Magnetic Moment'].info.format = '1.6g'
t['3822 BC'].info.format = '4.1f'
t['4004 BC'].info.format = '4.1f'
t['5554 BC'].info.format = '4.1f'
t['5665 BC'].info.format = '4.1f'
t.show_in_notebook()

idx,k,Creation Magnetic Moment,3822 BC,4004 BC,5554 BC,5665 BC
,,J / T,,,,
0,0.25,1.40804e+24,2029.2,2092.3,2630.4,2669.0
1,0.50,2.81609e+24,1635.6,1686.5,2120.2,2151.3
2,0.75,4.22413e+24,1468.9,1514.7,1904.2,1932.1
3,1.00,5.63218e+24,1369.9,1412.5,1775.8,1801.8


## D. Russell Humphreys, The Creation of the Earth's Magnetic Field, Creation Research Society Quarterly Vol 20, Sept 1983

In [7]:
# page 90
hmuP = 1.41e-26 * u.J / u.T
ldiff = muP - (muP.uncertainty * muP.unit) - hmuP
hdiff = muP + (muP.uncertainty * muP.unit) - hmuP
print('{0:.37f}'.format(hmuP))
print('{0:.37f}'.format(muP))
print('{0:.37f}'.format(ldiff))
print('{0:.37f}'.format(hdiff))
print('{0:.37f}'.format(hdiff-ldiff))

0.0000000000000000000000000141000000000 J / T
0.0000000000000000000000000141060679736 J / T
0.0000000000000000000000000000060679676 J / T
0.0000000000000000000000000000060679796 J / T
0.0000000000000000000000000000000000120 J / T


In [8]:
k = 0.25
hM_water = 2.992e-26 * u.kg
print('{0:.37f} my calc water mass'.format(M_water))
print('{0:.37f} Humphreys water mass'.format(hM_water))
hM_earth = 5.976e24 * u.kg
print('{0:.0f} CODATA 2018 Earth mass'.format(M_earth))
print('{0:.0f} Humphreys Earth mass'.format(hM_earth))
hn = hM_earth / hM_water

0.0000000000000000000000000299151924838 kg my calc water mass
0.0000000000000000000000000299200000000 kg Humphreys water mass
5972167867791379215155200 kg CODATA 2018 Earth mass
5976000000000000419430400 kg Humphreys Earth mass


In [9]:
n = M_earth / M_water
max_mu_earth_creation = 2 * n * muP
three_mu_earth_creation = 2 * 0.75 * n * muP
mu_earth_creation = 2 * k * n * muP
hmu_earth_creation = 2 * k * hn * hmuP
print('{0:.0f} Magnetic moment at creation'.format(mu_earth_creation))
print('{0:.0f} Humphreys magnetic moment at creation'.format(hmu_earth_creation))
print('{0:.0f} k=0.75 magnetic moment at creation'.format(three_mu_earth_creation))
print('{0:.0f} Max magnetic moment at creation'.format(max_mu_earth_creation))

1408043854947722987569152 J / T Magnetic moment at creation
1408114973262032180084736 J / T Humphreys magnetic moment at creation
4224131564843169768013824 J / T k=0.75 magnetic moment at creation
5632175419790891950276608 J / T Max magnetic moment at creation


## D. Eisenberg and W. Kauzman, The Structure and Property of Water, 1969 https://archive.org/details/structurepropert0000deis

In [10]:
t = 2049 
low_t = 2049 - 79
high_t = 2049 + 79
M0 = 2.088e23 
M = M0 * math.exp(-4000 / t)
current_mu = 7.9e22
years = 4000 + 2000

In [11]:
creation = current_mu / math.exp(-years / t) * u.J / u.T
low_creation = current_mu / math.exp(-years / low_t) * u.J / u.T
high_creation = current_mu / math.exp(-years / high_t) * u.J / u.T
print(low_creation)
print(creation)
print(high_creation)

1.6609302760744483e+24 J / T
1.4769073309232425e+24 J / T
1.3247734261251525e+24 J / T


In [12]:
def creation_mu(years):
    return current_mu / math.exp(-years / t) * u.J / u.T

In [13]:
print(creation_mu(7500))
print(creation_mu(8000))
print(creation_mu(10000))

3.0710351335825645e+24 J / T
3.9197824535543413e+24 J / T
1.040328946948061e+25 J / T


In [14]:
def creation_mu_k(years):
    creation_mu = current_mu / math.exp( -years / t) * u.J / u.T
    k = creation_mu / (2 * n * muP)
    return (k, creation_mu)

creation_mu_k(6023)

(<Quantity 0.26518687>, <Quantity 1.49357899e+24 J / T>)

In [15]:
creation_mu_k(7576)

(<Quantity 0.56587062>, <Quantity 3.18708258e+24 J / T>)

In [16]:
creation_mu_k(8000)

(<Quantity 0.69596242>, <Quantity 3.91978245e+24 J / T>)

In [17]:
creation_mu_k(10000)

(<Quantity 1.84711744>, <Quantity 1.04032895e+25 J / T>)

# IGRF Data

International Geomagnetic Reference Field: the 13th generation, Alken, P., Thébault, E., Beggan, C.D. et al. International Geomagnetic Reference Field: the thirteenth generation. Earth Planets Space 73, 49 (2021). doi: 10.1186/s40623-020-01288-x

https://www.ncei.noaa.gov/products/international-geomagnetic-reference-field

In [94]:
"""
I made two changes in the header line to align the columns:
(1) I replaced "g/h" with "gh" since the first column had only two characters.
(2) I added a space between "n" and "m" to make sure the rest of the columns aligned properly.

The first three lines of the file are skipped, but here they are:

# 13th Generation International Geomagnetic Reference Field Schmidt semi-normalised spherical harmonic coefficients, degree n=1,13
# in units nanoTesla for IGRF and definitive DGRF main-field models (degree n=1,8 nanoTesla/year for secular variation (SV))
c/s deg ord IGRF IGRF   IGRF   IGRF   IGRF   IGRF   IGRF   IGRF   IGRF   DGRF   DGRF   DGRF   DGRF   DGRF   DGRF   DGRF   DGRF   DGRF   DGRF   DGRF     DGRF      DGRF      DGRF      DGRF     IGRF      SV

"""

# Input the table from a text file
IGRF13 = QTable.read("igrf13coeffsmodified.txt", 
                  format='ascii.fixed_width', 
                  header_start=1, 
                  data_start=2,
                  col_starts=(0,2,4,8,15,22,29,36,43,50,57,64,71,78,85,92,99,106,113,120,127,134,141,148,157,167,177,187,196),
                  delimiter = ' ') 

# Define the unit for the following columns
IGRF13['n'].unit = u.dimensionless_unscaled
IGRF13['m'].unit = u.dimensionless_unscaled
IGRF13['2020-25'].unit = u.nT / u.year
for i in IGRF13.colnames[3:len(IGRF13.colnames)-1]:
    IGRF13[i].unit = u.nT

# Add meta data information
IGRF13.meta = {'Name': 'IGRF13'}

# Display the initial and ending portion of the table.  
# Uncomment "IGRF13.show_in_notebook()" and comment out "IGRF13" to display all rows with index column.
# IGRF13.show_in_notebook()
print(IGRF13.meta['Name'])
IGRF13

IGRF13


gh,n,m,1900.0,1905.0,1910.0,1915.0,1920.0,1925.0,1930.0,1935.0,1940.0,1945.0,1950.0,1955.0,1960.0,1965.0,1970.0,1975.0,1980.0,1985.0,1990.0,1995.0,2000.0,2005.0,2010.0,2015.0,2020.0,2020-25
,,,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT,nT / yr
str1,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
g,1.0,0.0,-31543.0,-31464.0,-31354.0,-31212.0,-31060.0,-30926.0,-30805.0,-30715.0,-30654.0,-30594.0,-30554.0,-30500.0,-30421.0,-30334.0,-30220.0,-30100.0,-29992.0,-29873.0,-29775.0,-29692.0,-29619.4,-29554.63,-29496.57,-29441.46,-29404.8,5.7
g,1.0,1.0,-2298.0,-2298.0,-2297.0,-2306.0,-2317.0,-2318.0,-2316.0,-2306.0,-2292.0,-2285.0,-2250.0,-2215.0,-2169.0,-2119.0,-2068.0,-2013.0,-1956.0,-1905.0,-1848.0,-1784.0,-1728.2,-1669.05,-1586.42,-1501.77,-1450.9,7.4
h,1.0,1.0,5922.0,5909.0,5898.0,5875.0,5845.0,5817.0,5808.0,5812.0,5821.0,5810.0,5815.0,5820.0,5791.0,5776.0,5737.0,5675.0,5604.0,5500.0,5406.0,5306.0,5186.1,5077.99,4944.26,4795.99,4652.5,-25.9
g,2.0,0.0,-677.0,-728.0,-769.0,-802.0,-839.0,-893.0,-951.0,-1018.0,-1106.0,-1244.0,-1341.0,-1440.0,-1555.0,-1662.0,-1781.0,-1902.0,-1997.0,-2072.0,-2131.0,-2200.0,-2267.7,-2337.24,-2396.06,-2445.88,-2499.6,-11.0
g,2.0,1.0,2905.0,2928.0,2948.0,2956.0,2959.0,2969.0,2980.0,2984.0,2981.0,2990.0,2998.0,3003.0,3002.0,2997.0,3000.0,3010.0,3027.0,3044.0,3059.0,3070.0,3068.4,3047.69,3026.34,3012.2,2982.0,-7.0
h,2.0,1.0,-1061.0,-1086.0,-1128.0,-1191.0,-1259.0,-1334.0,-1424.0,-1520.0,-1614.0,-1702.0,-1810.0,-1898.0,-1967.0,-2016.0,-2047.0,-2067.0,-2129.0,-2197.0,-2279.0,-2366.0,-2481.6,-2594.5,-2708.54,-2845.41,-2991.6,-30.2
g,2.0,2.0,924.0,1041.0,1176.0,1309.0,1407.0,1471.0,1517.0,1550.0,1566.0,1578.0,1576.0,1581.0,1590.0,1594.0,1611.0,1632.0,1663.0,1687.0,1686.0,1681.0,1670.9,1657.76,1668.17,1676.35,1677.0,-2.1
h,2.0,2.0,1121.0,1065.0,1000.0,917.0,823.0,728.0,644.0,586.0,528.0,477.0,381.0,291.0,206.0,114.0,25.0,-68.0,-200.0,-306.0,-373.0,-413.0,-458.0,-515.43,-575.73,-642.17,-734.6,-22.4


In [175]:
"""
This procedure creates a summary table from the IGRF data.

INPUT:
  t: The name of the IGRF table
  form: A format for the data columns with default 
  humphreys: A flag to tell which constants are being used.

"""           
def makesummarytable(t, form='5.0f', humphreys=False):
    # Separate calculation for each harmonics
    def hfactor(harmonics):
        t1 = (earthR / earthcoreR) ** ((2 * harmonics) + 1)
        t2 = (2 * math.pi * (earthR ** 3)) / m
        t3 = (harmonics + 1) / ((2 * harmonics) + 1)       
        return t1 * t2 * t3 / 1e24

    # Construct data for each harmonics in a column
    def makecol(col):
        return [hfactor(1) * np.sum(col[0:3]**2),
                hfactor(2) * np.sum(col[3:8]**2),
                hfactor(3) * np.sum(col[8:15]**2),
                hfactor(4) * np.sum(col[15:24]**2),
                hfactor(5) * np.sum(col[24:35]**2),
                hfactor(6) * np.sum(col[35:48]**2),
                hfactor(7) * np.sum(col[48:63]**2),
                hfactor(8) * np.sum(col[63:80]**2),
                hfactor(9) * np.sum(col[80:99]**2),
                hfactor(10) * np.sum(col[99:120]**2),
                hfactor(11) * np.sum(col[121:143]**2),
                hfactor(12) * np.sum(col[144:168]**2),
                hfactor(13) * np.sum(col[169:195]**2)
               ]
            
    # Define names for the rows
    rownames = ['Dipole Energy',
                'Quadrupole E',
                'Octopole E',
                '4th Harmonic',
                '5th Harmonic',
                '6th Harmonic',
                '7th Harmonic',
                '8th Harmonic',
                '9th Harmonic',
                '10th Harmonic',
                '11th Harmonic',
                '12th Harmonic',
                '13th Harmonic'
               ]

    # Construct the columns
    if humphreys:
        earthR = R_earth_H
        earthcoreR = R_earth_core_H
        m = mu0_H
        humphreysused = 'Yes'
    else:
        earthR = R_earth.to(u.km)
        earthcoreR = R_earth_core_H
        m = mu0
        humphreysused = 'No'
    cols = [rownames]
    for i in t.colnames[3:len(t.colnames)-1]:
        cols.append(makecol(t[i]))

    # Construct the names of the columns
    colnames = ['Years']
    for i in t.colnames[3:len(t.colnames)-1]:
        colnames.append(i)

    # Construct the metadata
    metadata = {'Name' : 'IGRF Summary Data',
                'Humphreys Used' : humphreysused,
                'Earth Radius' : earthR,
                'Earth Core Radius' : earthcoreR,
                'Vacuum Magnetic Permeability' : m
               }

    # Create the table
    s = QTable(cols, names=colnames, meta=metadata)

    # Format the columns for printing
    for i in s.colnames[1:len(s.colnames)]:
        s[i].info.format = form
        
    return s

"""
A procedure to return the metadata for a table.

INPUT:
t : The name of the table which contains metadata.
"""
def displaymetadata(t):
    for key in t.meta:
        if key in ('Name','Humphreys Used'):
            print('{} : {}'.format(key, t.meta[key]))
        else:
            print()
            print(t.meta[key])


In [167]:
Summary13 = makesummarytable(IGRF13, humphreys=True)
Summary13

Years,1900.0,1905.0,1910.0,1915.0,1920.0,1925.0,1930.0,1935.0,1940.0,1945.0,1950.0,1955.0,1960.0,1965.0,1970.0,1975.0,1980.0,1985.0,1990.0,1995.0,2000.0,2005.0,2010.0,2015.0,2020.0
,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H,km3 nT2 m / H
str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
Dipole Energy,5520,5492,5455,5406,5354,5308,5268,5238,5218,5198,5184,5166,5138,5108,5067,5024,4984,4938,4901,4867,4837,4809,4782,4756,4736
Quadrupole E,196,202,209,216,222,228,235,242,250,260,270,279,289,297,306,317,331,345,357,370,383,396,410,426,444
Octopole E,285,291,299,311,324,338,352,367,383,399,412,424,432,440,446,453,462,470,479,488,492,493,493,496,503
4th Harmonic,312,318,324,330,334,337,341,345,352,372,377,374,373,369,365,359,351,345,341,339,333,328,323,318,313
5th Harmonic,159,156,154,152,150,148,148,149,151,159,164,172,178,179,185,189,191,189,188,186,185,185,185,186,189
6th Harmonic,157,160,161,163,166,169,170,173,175,171,170,169,166,161,150,142,137,131,127,124,120,114,105,95,88
7th Harmonic,95,95,95,96,96,95,96,95,93,78,80,85,89,98,101,104,113,118,120,118,120,121,123,126,127
8th Harmonic,33,33,33,33,34,35,35,36,37,38,60,49,49,42,40,41,46,50,52,61,62,64,66,66,63


In [176]:
displaymetadata(Summary13)

Name : IGRF Summary Data
Humphreys Used : Yes

  Name   = Humphreys' Earth radius
  Value  = 6371.2
  Uncertainty  = 0
  Unit  = km
  Reference = D. Russell Humphreys, The Earth's Magnetic Field is Still Losing Energy, CRSQ 39, 2002, p.6

  Name   = Humphreys' Earth radius core
  Value  = 3471
  Uncertainty  = 0
  Unit  = km
  Reference = D. Russell Humphreys, The Earth's Magnetic Field is Still Losing Energy, CRSQ 39, 2002, p.8

  Name   = Humphreys' vacuum magnetic permeability
  Value  = 1.2566370614359173e-06
  Uncertainty  = 0
  Unit  = H / m
  Reference = D. Russell Humphreys, The Creation of Earth's Magnetic Field, CRSQ 20, 1983, p.92


# IGRF9 Data

In [154]:
IGRF9 = QTable.read("igrf9coeffs.txt", 
                  format='ascii.csv', 
                  header_start=0, 
                  data_start=1,
                  delimiter = '\t') 
IGRF9

g/h,n,m,1900.0,1905.0,1910.0,1915.0,1920.0,1925.0,1930.0,1935.0,1940.0,1945.0,1950.0,1955.0,1960.0,1965.0,1970.0,1975.0,1980.0,1985.0,1990.0,1995.0,2000.0,SV
str1,int32,int32,float64,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,int32,float64,float64
g,1,0,-31543.0,-31464,-31354,-31212,-31060,-30926,-30805,-30715,-30654,-30594,-30554,-30500,-30421,-30334,-30220,-30100,-29992,-29873,-29775,-29692,-29619.4,13.3
g,1,1,-2298.0,-2298,-2297,-2306,-2317,-2318,-2316,-2306,-2292,-2285,-2250,-2215,-2169,-2119,-2068,-2013,-1956,-1905,-1848,-1784,-1728.2,11.6
h,1,1,5922.0,5909,5898,5875,5845,5817,5808,5812,5821,5810,5815,5820,5791,5776,5737,5675,5604,5500,5406,5306,5186.1,-21.2
g,2,0,-677.0,-728,-769,-802,-839,-893,-951,-1018,-1106,-1244,-1341,-1440,-1555,-1662,-1781,-1902,-1997,-2072,-2131,-2200,-2267.7,-14.4
g,2,1,2905.0,2928,2948,2956,2959,2969,2980,2984,2981,2990,2998,3003,3002,2997,3000,3010,3027,3044,3059,3070,3068.4,-3.7
h,2,1,-1061.0,-1086,-1128,-1191,-1259,-1334,-1424,-1520,-1614,-1702,-1810,-1898,-1967,-2016,-2047,-2067,-2129,-2197,-2279,-2366,-2481.6,-22.7
g,2,2,924.0,1041,1176,1309,1407,1471,1517,1550,1566,1578,1576,1581,1590,1594,1611,1632,1663,1687,1686,1681,1670.9,-3.6
h,2,2,1121.0,1065,1000,917,823,728,644,586,528,477,381,291,206,114,25,-68,-200,-306,-373,-413,-458.0,-11.1
g,3,0,1022.0,1037,1058,1084,1111,1140,1172,1206,1240,1282,1297,1302,1302,1297,1287,1276,1281,1296,1314,1335,1339.6,-1.1


In [155]:
Summary9 = makesummarytable(IGRF9, humphreys=True)
Summary9

Years,1900.0,1905.0,1910.0,1915.0,1920.0,1925.0,1930.0,1935.0,1940.0,1945.0,1950.0,1955.0,1960.0,1965.0,1970.0,1975.0,1980.0,1985.0,1990.0,1995.0,2000.0
,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H,km3 m / H
str13,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
Dipole Energy,5520,5492,5455,5406,5354,5308,5268,5238,5218,5198,5184,5166,5138,5108,5067,5024,4984,4938,4901,4867,4837
Quadrupole E,196,202,209,216,222,228,235,242,250,260,270,279,289,297,306,317,331,345,357,370,383
Octopole E,285,291,299,311,324,338,352,367,383,399,412,424,432,440,446,453,462,470,479,488,492
4th Harmonic,312,318,324,330,334,337,341,345,352,372,377,374,373,369,365,359,351,345,341,339,333
5th Harmonic,159,156,154,152,150,148,148,149,151,159,164,172,178,179,185,189,191,189,188,186,185
6th Harmonic,157,160,161,163,166,169,170,173,175,171,170,169,166,161,150,142,137,131,127,124,120
7th Harmonic,95,95,95,96,96,95,96,95,93,78,80,85,89,98,101,104,113,118,120,118,120
8th Harmonic,33,33,33,33,34,35,35,36,37,38,60,49,49,42,40,41,46,50,52,61,62
